# Import timeframe data

In [51]:
import numpy as np

# Import data

In [52]:
games = np.load('/tmp/game_data.npy')
print (games.shape)

(49996, 6, 30)


# Look at data

In [53]:
print(games[1][4])

[ 0.12135036  0.          0.23256705  0.33121387  0.52941176  0.32201646
  0.32492818  0.23529412  0.19783307  0.17782027  0.18181818  0.18721634
  0.37152581  0.47222222  0.63304479  0.33768826  0.2972973   0.25397681
  0.20527859  0.29032258  0.35419682  0.08521999  0.          0.12058347
  0.29606099  0.35135135  0.3977591   0.37237533  0.39473684  0.54598145]


In [54]:
labels = np.load('/tmp/game_labels.npy')
print (labels.shape)

(49996, 2)


# Separate train and test data

In [56]:

#labels_binary_length_corrected = labels.reshape((300000, 2))
labels_binary_length_corrected = labels

number_of_features_to_use = games.shape[2]

data = np.zeros((games.shape[0],games.shape[1], number_of_features_to_use))
for i in range(games.shape[0]):
    for j in range(games.shape[1]):
        data[i, j] = games[i,j][:number_of_features_to_use]
print (data.shape)


num_train_examples = int(len(data) * 0.9)
print(data.shape)
x_train = data[:num_train_examples,:]
y_train = labels_binary_length_corrected[:num_train_examples,:]
x_test = data[num_train_examples:,:]
y_test = labels_binary_length_corrected[num_train_examples:,:]

#x_train = np.reshape(x_train, x_train.shape + (1,))
#x_test = np.reshape(x_test, x_test.shape + (1,))

print(x_train.shape)

(49996, 6, 30)
(49996, 6, 30)
(44996, 6, 30)


In [57]:
print(x_train[0][1])

[ 0.31389102  0.          0.06256207  0.10334789  0.1         0.09553159
  0.13343924  0.          0.13034411  0.1618123   0.          0.16273187
  0.1503876   0.125       0.11660448  0.14111922  0.25        0.06569966
  0.11120064  0.125       0.05201342  0.25842697  0.75        0.29274395
  0.0748503   0.          0.06155076  0.50081699  0.125       0.11210762]


In [58]:
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.regularizers import l2
import keras

model = Sequential()

model.add(LSTM(
    input_shape=(None, x_train.shape[2]),
    units=50,
    return_sequences=False)
    )
#model.add(Dropout(0.1))


model.add(Dense(
    units=2))
model.add(Activation('softmax'))
#model.compile(loss='mse', optimizer='rmsprop')
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.adam(lr=0.001), metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])
#model.compile(loss=keras.losses.categorical_crossentropy,
#             optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.0, nesterov=False),
#             metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])

In [59]:
res = model.fit(x_train, y_train, epochs=5, batch_size=1000, validation_split=0.2)

Train on 35996 samples, validate on 9000 samples
Epoch 1/5
35996/35996 [==============================] - 4s - loss: 0.6894 - mean_absolute_error: 0.4969 - categorical_accuracy: 0.5366 - val_loss: 0.6816 - val_mean_absolute_error: 0.4928 - val_categorical_accuracy: 0.5796

In [50]:
# Test loss and accuracy
loss_and_metrics = model.evaluate(x_test, y_test)
print(loss_and_metrics)

4928/5000 [============================>.] - ETA: 0s[9.1699713287353521, 23749.512900000002, 1.0]
